#  Apply affine transformation for registering sMRI to dMRI using DiPy

BMED360-2021  `04-dmri-do-affine-reg-anat2dwi.ipynb`

(assuming the `02-dmri-find-affine-fs-brainmask2dwi.ipynb` and `03-dmri-reconstruction-dti.ipynb` notebooks have been executed)

<a href="https://colab.research.google.com/github/computational-medicine/BMED360-2021/blob/main/Lab3-diffusion-MRI/04-dmri-do-affine-reg-anat2dwi.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Learning objectives

- **Image registration between Freesurfer segmenations (in native space) and diffusion data using Dipy**

Assuming `02-dmri-find-affine-fs-brainmask2dwi.ipynb` is run

Also assume that the images in the Freesurfer tree is transformed from Freesurfer space to native space using mri_vol2vol, e.g. 
- `sub_101_tp1_orig_in_native_space.nii.gz`, 
- `sub_101_tp1_brain_in_native_space.nii.gz`, 
- `sub_101_tp1_brainmask_in_native_space.nii.gz`, 
- `sub_101_tp1_ribbon_in_native_space.nii.gz`, 
- `sub_101_tp1_aseg_in_native_space.nii.gz`,
- `sub_101_tp1_wmparc_in_native_space.nii.gz`,
- `sub_101_tp1_aparc+aseg_in_native_space.nii.gz`,


(see end of  `03-diff-dipy-find-affine-brainmask2dwi.ipynb`)

### For using Colab
**--> (some of) the following libraries must be `pip installed` (i.e. uncommet these among the following pip commands):**

In [1]:
#!pip install gdown
#!pip install nilearn
#!pip install dipy

**Download a data file from Google Drive using gdown** (https://github.com/wkentaro/gdown)

In [2]:
import gdown
import shutil
import sys
import os
from os.path import expanduser, join, basename, split
import glob
import shutil
import platform

Check your platform for running this notebook

In [3]:
if platform.system() == 'Darwin':
    print(f'OK, you are running on MacOS ({platform.version()})')
if platform.system() == 'Linux':
    print(f'OK, you are running on Linux ({platform.version()})')
if platform.system() == 'Windows':
    print(f'OK, but consider to install WSL for Windows10 since you are running on {platform.system()}')
    print('Check https://docs.microsoft.com/en-us/windows/wsl/install-win10')

OK, you are running on MacOS (Darwin Kernel Version 20.4.0: Fri Mar  5 01:14:14 PST 2021; root:xnu-7195.101.1~3/RELEASE_X86_64)


In [4]:
cwd = os.getcwd()

In [5]:
working_dir = join(cwd, 'data')
bids_dir = '%s/bids_bg_bmed360' % (working_dir)
dmri_res = '%s/dmri_results' % (working_dir)

In [6]:
# Download zip-file if ./data does not exist (as when running in Colab)

if os.path.isdir(working_dir) == False:
    
    ## Download data.zip for Google Drive            
    # https://drive.google.com/file/d/1pX6Mx_9P8fojDXmbTz-th5FKFYZDVFuO/view?usp=sharing
    file_id = '1pX6Mx_9P8fojDXmbTz-th5FKFYZDVFuO'
    url = 'https://drive.google.com/uc?id=%s' % file_id
    output = './data.zip'
    gdown.download(url, output, quiet=False)
    
    ## Unzip the assets file into `./data`
    shutil.unpack_archive(output, '.')
    
    ## Delete the `data.zip` file
    os.remove(output)
else:
    print(f'./data  exists already!')

./data  exists already!


In [7]:
# Download zip-file if ./data/dmri_results does not exist (as when running in Colab)

if os.path.isdir(dmri_res) == False:
    
    ## Download dmri_results.zip for Google Drive            
    # https://drive.google.com/file/d/1pX6Mx_9P8fojDXmbTz-th5FKFYZDVFuO/view?usp=sharing
    file_id = '1wu5pzAcE2hyZymq-IzuzKYGK_lMYbnJy'
    url = 'https://drive.google.com/uc?id=%s' % file_id
    output = 'dmri_results.zip'
    gdown.download(url, output, quiet=False)
    
    ## Unzip the assets file into `./data/dmri_results`
    shutil.unpack_archive(output, './data/')
    
    ## Delete the `dmri_results.zip` file
    os.remove(output)
else:
    print(f'./data/dmri_results  exists already!')

./data/dmri_results  exists already!


## Import libraries

In [8]:
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import expanduser, join, basename, split
import time
from dipy.viz import regtools
from dipy.align.imaffine import (AffineMap, MutualInformationMetric, AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D, RigidTransform3D, AffineTransform3D)
from dipy.io.image import save_nifti

/Users/arvid/opt/anaconda3/envs/bmed360/lib/python3.8/site-packages/dipy/viz/__init__.py:31: UserWarning: You do not have FURY installed. Some visualization functions might not work for you. For installation instructions, please visit: https://fury.gl/
  warnings.warn(


## Make an affine registration when the affine map is computed previuously 

In [9]:
def do_affine_reg_anat2dwi(inpdir, sub, ses, outdir):
    r"""
    Affine registration between a Freesurfer segmented region or mask and a mean S0 in native DWI space using 
    multilevel optimization and the Mutual Information similarity measure. 
    Based on the affine map obtained from '02-dmri-find-affine-fs-brainmask2dwi.ipynb'

    Parameters
    ----------
    inpdir     : input directory e.g. inpdir = './data/dmri_results' where the moving files from
                    Freesurfer segmentation mapped to T1w native space are stored.
    sub        : subject id, e.g. 102 for sub-102
    ses        : session number, eg 1 for ses-1
    outdir     : output directory e.g. outdir = inpdir, where resulting affinely transformed input moving image 
                    are stored,

    Returns
    -------
    NIL


    Notes
    -----
    
    The affine map is saved by '02-dmri-find-affine-fs-brainmask2dwi.ipynb':
        np.savetxt(affine_fn, affine.get_affine(), delimiter=',')  
    and loaded by:  
        aff = np.loadtxt(affine_fn, delimiter=',')
        affine_map = AffineMap(np.eye(4))
        affine_map.set_affine(aff)

    """
    
    # Preparations

    # Load static image in DWI native space
    static_fn = join(inpdir, 'sub_%d_tp%d_dwi_S0_mean.nii.gz' % (sub, ses))
    static_img= nib.load(static_fn)
    static_data = static_img.get_fdata()
    static_affine = static_img.affine
    
    # Load affine map: 
    affine_fn = join(inpdir, 'sub_%d_tp%d_affine_brainmask_to_dwi_S0_mean.mat' % (sub, ses))
    aff = np.loadtxt(affine_fn, delimiter=',')
    #affine_map = AffineMap(np.eye(4))
    #affine_map.set_affine(aff)
    
    
    # Load Freesurfer-segmented results in native anatomical space
    
    targ_name = 'native_space'
    
    # Images (linear interpolation)
    fs_img = ['orig', 'brain']
    for i, ima in enumerate(fs_img):
        moving_ima_fn = join(inpdir,'sub_%d_tp%d_%s_in_%s.nii.gz' % (sub, ses, ima, targ_name))
        moving_ima = nib.load(moving_ima_fn)
        moving_ima_data = moving_ima.get_fdata()
        moving_ima_affine = moving_ima.affine
        
        affine_map = AffineMap(affine=np.eye(4),    # identity
                           domain_grid_shape=static_data.shape, 
                           domain_grid2world=static_affine,
                           codomain_grid_shape=moving_ima_data.shape, 
                           codomain_grid2world=moving_ima_affine)
        affine_map.set_affine(aff)
        
        transformed_ima = affine_map.transform(
        image=moving_ima_data,
        interpolation='linear', # 'linear' (k-linear interpoltaion) or 'nearest'
        image_grid2world=None,
        sampling_grid_shape=None,
        sampling_grid2world=None,
        resample_only=False
        )
    
        transformed_ima_fn = join(outdir, 'sub_%d_tp%d_%s_in_native_space_aff_to_dwi_S0_mean.nii.gz' % \
                                  (sub, ses, ima))
        moving2static_ima = nib.Nifti1Image(transformed_ima, static_affine)
        nib.save(moving2static_ima, transformed_ima_fn)
    
    
    # Masks (nearest neighbour interpolation)
    fs_msk = ['brainmask', 'ribbon', 'aseg', 'wmparc', 'aparc+aseg']
    for i, segm in enumerate(fs_msk):
        moving_mask_fn = join(inpdir,'sub_%d_tp%d_%s_in_%s.nii.gz' % (sub, ses, segm, targ_name))
        moving_mask = nib.load(moving_mask_fn)
        moving_mask_data = moving_mask.get_fdata()
        moving_mask_affine = moving_mask.affine
        
        affine_map = AffineMap(affine=np.eye(4),    # identity
                           domain_grid_shape=static_data.shape, 
                           domain_grid2world=static_affine,
                           codomain_grid_shape=moving_mask_data.shape, 
                           codomain_grid2world=moving_mask_affine)
        affine_map.set_affine(aff)
        
        transformed_mask = affine_map.transform(
        image=moving_mask_data,
        interpolation='nearest', # 'linear' (k-linear interpoltaion) or 'nearest'
        image_grid2world=None,
        sampling_grid_shape=None,
        sampling_grid2world=None,
        resample_only=False
        )
    
        transformed_mask_fn = join(outdir, 'sub_%d_tp%d_%s_in_native_space_aff_to_dwi_S0_mean.nii.gz' % \
                                   (sub, ses, segm))
        moving2static_mask = nib.Nifti1Image(transformed_mask, static_affine)
        nib.save(moving2static_mask, transformed_mask_fn)
           
    
    return


In [10]:
#do_affine_reg_anat2dwi?

In [11]:
inp_dir = dmri_res 
outp_dir = inp_dir

### Test the function

In [12]:
sub = 102
ses = 1

In [13]:
%%time
do_affine_reg_anat2dwi(inp_dir, sub, ses, outp_dir)

CPU times: user 5.29 s, sys: 265 ms, total: 5.56 s
Wall time: 1.86 s


## Run for all subjects

In [14]:
%%time

for sub in [102, 103, 111, 123]:
    for ses in [1]:
        
        print(f'Computing sub:{sub} ses:{ses}')
        do_affine_reg_anat2dwi(inp_dir, sub, ses, outp_dir)


Computing sub:102 ses:1
Computing sub:103 ses:1
Computing sub:111 ses:1
Computing sub:123 ses:1
CPU times: user 22.7 s, sys: 812 ms, total: 23.6 s
Wall time: 7.16 s
